In [1]:
#Import Required Libraries:
import pandas as pd
import networkx as nx
import json

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/umassdgithub/Fall-2023-DataViz/main/Week-8-ForceSimulator/data/data_scopus.csv")


In [3]:
# Dropping the columns which do not have values for the below
df = df.dropna(subset=['Author(s) ID','Year','Authors','Authors with affiliations'])
df=df.fillna(0)

In [4]:
#Creating nodes
G = nx.Graph()
nodes = {}
for _, row in df.iterrows():
    # Extract authors from the "Authors" column
    authors = row['Authors'].split(',')
    author_Id = row['Author(s) ID'].split(';')
    Title=row['Title']
    Year=row['Year']
    Citations=row['Cited by']
    Publisher=row['Publisher']
    auth_w_aff=row['Authors with affiliations']

    # Adding authors as nodes

    for author in range(len(authors)):

        aid= author_Id[author]
        auth_name= ';'.join(authors)
        title=Title

        if(aid!=""):
         nodes={'id':aid,
          "Authors": auth_name,
          "Title": title,
          "Year": Year,
          "Citations": Citations,
          "Publisher": Publisher,
          "Author with affiliations":auth_w_aff
           }
         G.add_node(aid,**nodes)
from networkx.readwrite import json_graph
with open("publication_network.json", "w") as f:
    json.dump(json_graph.node_link_data(G), f)

In [5]:
#Edges
# Load the data from the JSON file
with open('publication_network.json', 'r') as infile:
    data = json.load(infile)
newnodes=[]
G=nx.Graph()
def get_author_country(authors_with_affiliations):
    first_affiliation = authors_with_affiliations.split(';')[0].strip()
    country = first_affiliation.split(',')[-1].strip()
    return country


In [6]:
for node_data in data['nodes']:
    eid = node_data.get('id')
    authors=node_data.get('Authors')
    title = node_data.get('Title')
    year = node_data.get('Year')
    citations = node_data.get('Citations')
    publisher = node_data.get('Publisher')

    authors_with_affiliations = str(node_data.get('Author with affiliations'))

    country = get_author_country(authors_with_affiliations)

    G.add_node(eid,authors=authors, title=title, year=year, citations=citations, publisher=publisher,authors_with_affiliations=authors_with_affiliations, country=country)


clusters = list(nx.connected_components(G))

#Assigning class
class_mapping = {node: idx for idx, cluster in enumerate(clusters) for node in cluster}
nx.set_node_attributes(G, class_mapping, 'class')

In [ ]:
coauthorship_dict = {}
for row in df.iterrows():
    authors = row[1]['Author(s) ID'].split(';')
    for i in range(len(authors)):
      for j in range(i+1, len(authors)):
        if(authors[i]!="" and authors[j]!=""):
          G.add_edge(authors[i], authors[j])

coauthorship_data = {'nodes': [{'id': node, **G.nodes[node]} for node in G.nodes()],'links': [{'source': source, 'target': target} for source, target in G.edges()]}

with open('authorsdata.json', 'w', encoding='utf-8') as outfile:
    json.dump(coauthorship_data, outfile, ensure_ascii=False)